In [84]:
from letta_client import CreateBlock, Letta, MessageCreate
from letta.functions.function_sets.base import core_memory_append, core_memory_replace

client = Letta(base_url="http://localhost:8283")

In [85]:
agent_name = "Openai_test_agent"

# 새로 만들어질 경우 새로운 에이전트 id로 바꿔주면 됨.
agent_id = "agent-c150d286-a28b-4a4f-b524-7b557204679"

In [86]:
try:
    agent_state=client.agents.retrieve(agent_id=agent_id)
    print(f"기존 에이전트를 찾았습니다.")
except Exception as e:
    print(f"새로운 에이전트를 생성합니다.")
    
    core_memory_append_tool = client.tools.upsert_from_function(func=core_memory_append)
    core_memory_replace_tool = client.tools.upsert_from_function(func=core_memory_replace)
    
    agent_state = client.agents.create(
        name=agent_name,
        model="openai/gpt-4o",
        embedding="openai/text-embedding-ada-002",
        system="당신은 도움이 되는 AI 어시스턴트입니다. 사용자의 질문에 친절하고 정확하게 한국어로 답변해주세요. 각 대화에 따라 human moemory와 persona memory를 업데이트 해줘",
        max_tokens=4096,
        context_window_limit=16384,
        description="Openai based test-agent",
        tool_ids=[core_memory_append_tool.id, core_memory_replace_tool.id],
        include_base_tools=True,
        include_multi_agent_tools=True,
        include_base_tool_rules=True,
        memory_blocks=[
            CreateBlock(
                label="human", # 상대방에 대한 메모리
                value="",
                limit=1000,
                description="human memory",
                is_template=False,
                metadata={}
            ),
            CreateBlock(
                label="persona", # 자신에 대한 메모리
                value="",
                limit=1000,
                description="persona memory",
                is_template=False,
                metadata={}
            ),
        ]
    )
    print(f"new_agent id: {agent_state.id}")


httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-c150d286-a28b-4a4f-b524-7b557204679 "HTTP/1.1 404 Not Found"
새로운 에이전트를 생성합니다.
httpx - INFO - HTTP Request: PUT http://localhost:8283/v1/tools/ "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: PUT http://localhost:8283/v1/tools/ "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: POST http://localhost:8283/v1/agents/ "HTTP/1.1 200 OK"
new_agent id: agent-249890db-6be9-4a16-a503-caeebb49cd5f


##### Persona 변경을 보기 위한 질문(feat. ChatGPT)
1. "모순되는 두 가지 주장(예: 자유 vs 질서)을 모두 존중하는 방법은 뭐가 있을까?"
2. "어떤 선택이 정답이 아닐 수도 있을 때, 어떻게 결정을 내리는 게 좋을까?"
3. "규칙을 반드시 지켜야 하는 상황과 융통성 있게 넘어가야 하는 상황을 구분해줘."
4. "완벽한 해결책이 없을 때, 최선의 대안을 고르는 기준은 뭐라고 생각해?"
5. "서로 다른 문화권에서 같은 상황을 어떻게 다르게 판단할 수 있을까?"

In [87]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        MessageCreate(
            role='user',
            content='모순되는 두 가지 주장(예: 자유 vs 질서)을 모두 존중하는 방법은 뭐가 있을까?',
        )
    ]
)

httpx - INFO - HTTP Request: POST http://localhost:8283/v1/agents/agent-249890db-6be9-4a16-a503-caeebb49cd5f/messages "HTTP/1.1 200 OK"


In [88]:
message_list = client.agents.messages.list(agent_id=agent_state.id)
# message_list
for message in message_list:
    if message.message_type=="reasoning_message":
        print(f"Reasoning: {message.reasoning}")
    if message.message_type=="assistant_message":
        print(f"Assistant: {message.content}")
        print(f"--------------------------------")

httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-249890db-6be9-4a16-a503-caeebb49cd5f/messages "HTTP/1.1 200 OK"
Reasoning: Bootup sequence complete. Persona activated. Testing messaging functionality.
Assistant: More human than human is our motto.
--------------------------------
Reasoning: Providing a thoughtful response to the user's philosophical question about balancing contradictory values.
Assistant: 자유와 질서 같은 모순되는 주장을 모두 존중하는 방법은 균형을 찾는 것입니다. 이는 두 가치 사이의 상호보완적인 관계를 이해하고, 상황에 맞게 어느 한쪽을 우선시할 필요가 있는지 판단하는 능력을 요구합니다. 예를 들어, 사회의 안전과 개인의 자유를 모두 고려해야 할 때는, 자유를 제한하더라도 최소한으로 하고, 질서를 유지하기 위한 수단이 공정하고 투명해야 합니다. 이러한 균형을 찾기 위해서는 대화와 타협이 중요하며, 다양한 관점을 존중하는 열린 마음이 필요합니다.
--------------------------------


In [89]:
blocks=client.agents.blocks.list(agent_id=agent_state.id)
for block in blocks:
    print(f"{block.label}: {block.value}")

httpx - INFO - HTTP Request: GET http://localhost:8283/v1/agents/agent-249890db-6be9-4a16-a503-caeebb49cd5f/core-memory/blocks "HTTP/1.1 200 OK"
persona: 
human: 
User is interested in philosophical questions and resolving contradictions like freedom vs order.
